In [1]:
import os
import pandas as pd

#Read in Liberal data from txt files
data = []

for f in os.listdir('./libdata'):
    with open('./libdata/' + f, 'r') as myfile:
        data.append(myfile.read())
        
srsLib = pd.Series(data, name = 'text')

#Read in Conservative data from txt files
data = []

for f in os.listdir('./condata'):
    with open('./condata/' + f, 'r') as myfile:
        data.append(myfile.read())

srsCon = pd.Series(data, name = 'text')

#len('Message-ID: <2262872.1075863600393.JavaMail.evans@thyme>')
#dfSpam = pd.concat([enron, pd.Series([0]*len(enron), name = 'isFood')], axis = 1)
#dfSpam.to_csv('enron.csv')
#dfSpam.head()

print (srsLib, srsCon)

0    SECTIONS HOME SEARCHSKIP TO CONTENTSKIP TO NAV...
1    SECTIONS HOME SEARCHSKIP TO CONTENTSKIP TO NAV...
2    SECTIONS HOME SEARCHSKIP TO CONTENTSKIP TO NAV...
3    SECTIONS HOME SEARCHSKIP TO CONTENTSKIP TO NAV...
Name: text, dtype: object 0    DOW JONES, A NEWS CORP COMPANY \nDJIA\n?\n1988...
1    \n \n       \nDailyCaller\n\n \n\nHOME\nPOLITI...
2    \n \n       \nDailyCaller\n\n \n\nHOME\nPOLITI...
3    \nPowered by Tynt\nshare on Facebooktweet this...
Name: text, dtype: object


In [3]:
import pandas as pd
articles = pd.read_csv('../scrape/articles_bias.csv')

In [4]:
sum(articles['bias_score'] == .5)

54

In [6]:
articles.head(5)
[articles['bias_score'] > .5]
articles = articles.loc[articles['bias_score'] != .5]
articles.loc[articles['bias_score'] > .5, 'bias_score'] = 1
articles.loc[articles['bias_score'] < .5 , 'bias_score'] = 0

In [8]:
articles['bias_score'].value_counts()

1.0    29
0.0    29
Name: bias_score, dtype: int64

In [9]:
articles[['text', 'bias_score']].head(5)

,text,bias_score
0,"Running for president, Donald Trump promised a...",0.0
2,Bill Clark/CQ Roll Call via Getty Images Forme...,0.0
8,"\n\nSenator Jeff Sessions, Donald Trump’s nomi...",0.0
9,Tillerson’s answer on Exxon’s dealings around ...,0.0
10,McMentum? New Utah poll has Trump ahead but Mc...,1.0


In [20]:
from nltk.tokenize import RegexpTokenizer

def clean_text(text):
    return ' '.join(
        RegexpTokenizer(r'\w+').tokenize(
        str.lower(
            text
            )
        )
    )

In [21]:
articles['text'] = articles['text'].apply(clean_text)

In [45]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

#from sklearn.cross_validation import KFold
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import GridSearchCV

import numpy as np

parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
               'tfidf__use_idf': (True, False),
               'clf__alpha': (1e-2, 1e-3),
}

X_train, y_train, X_test, y_test = train_test_split(articles['text'], articles['bias_score'], test_size=0.33, random_state=42)
#kf = cross_validation.KFold(4, n_folds=5)

text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
 ])

#print(cross_val_score(text_clf, articles['text'], articles['bias_score']))

text_clf.fit(articles['text'], articles['bias_score'])

# from sklearn.externals import joblib
# joblib.dump(text_clf, 'text_clf.pkl')

predicted = text_clf.predict_proba(y_train)
#np.mean(predicted == y_test)

In [56]:
list(zip(*predicted))[1] #These are the predictions in probability.

(0.35799712900046837,
 0.5638061241041824,
 0.59025101296778038,
 0.52803263267198297,
 0.38185393790870048,
 0.32485818576540559,
 0.5894825621331431,
 0.57626563026414068,
 0.61348095870146035,
 0.31749436756957511,
 0.63124362423183489,
 0.39145425046819227,
 0.34703251757002851,
 0.34135430451311938,
 0.53139869281825514,
 0.59544453165847866,
 0.38842925052167959,
 0.62579772822648949,
 0.55308558195643043,
 0.70654941027035267)

In [10]:
articles['bias_score'].value_counts()

0.0    83
1.0    29
Name: bias_score, dtype: int64

In [89]:
#Split into train and test set
import numpy as np
import sklearn
from sklearn.cross_validation import train_test_split

X = dfData['text']
y = dfData['isLib']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [90]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
 ])

In [91]:
text_clf.fit(X_train, y_train)

from sklearn.externals import joblib
joblib.dump(text_clf, 'text_clf.pkl')

predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)

0.33333333333333331